In [1]:
import os
import json
import pandas as pd
from functools import reduce
import sqlite3
import pandas as pd
from tqdm import tqdm

In [8]:
weather_station_dict = {
    "601307eb571de9001b1b6c6c-WetterBrG_": 'WetterBrg',
    "617ab96c8b4e03001b05b489-_Wald_der_Zukunft__Stadtbergen": 'Wald der Zukunft - Stadtbergen',
    "633315b01ad7df001b349a76-PoS": 'PoS'
}

data_file_name = "data.csv"

renaming_dict = {
    "BeleuchtungsstÃ¤rke lx": "Licht Lux",
    "Luftdruck hPa": "Luftdruck hPa (=mBar)",
    "PM10 Âµg/mÂ³": "PM10 Âµm",
    "PM2.5 Âµg/mÂ³": "PM2.5 Âµm",
    "UV-IntensitÃ¤t Î¼W/cmÂ²": "UV Index"
}

column_dict = {
    "Temperatur Â°C": "Temperatur °C",
    "Bodentemperatur Â°C": "Bodentemperatur °C",
    "PM2.5 Âµm": "PM2.5 µm",
    "PM10 Âµm": "PM10 µm"
}

db_name = "sensor_daten.sqlite"

In [9]:
def get_assignment(assignment_path, file_paths): 
    assignment_dict = json.load(open(assignment_path))

    # With assingment
    sensor_names = assignment_dict.get("sensors")
    assignment = dict()
    for sensor in sensor_names:
        assignment[sensor.get('_id')] = {'title': sensor.get('title') + " " + sensor.get('unit')}

    for _id, _dict in assignment.items():
        for file in file_paths:
            if _id in file:
                _dict['file'] = file
                break

    return assignment

In [10]:
def create_database():
    """Create an SQLite database and table."""
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS WeatherData (
            createdAt TEXT,
            Temperatur REAL,
            rel_Luftfeuchte REAL,
            Luftdruck REAL,
            Bodenfeuchte REAL,
            Bodentemperatur REAL,
            Licht REAL,
            UV_Index REAL,
            PM25 REAL,
            PM10 REAL,
            station TEXT
        )''')
    
    conn.commit()
    conn.close()

def insert_dataframe_to_db(df, db_name):
    """Insert a pandas DataFrame into the SQLite database."""
    conn = sqlite3.connect(db_name)
    
    # Clean the DataFrame column names
    df.columns = [
        'createdAt', 'Temperatur', 'rel_Luftfeuchte', 'Luftdruck', 'Bodenfeuchte', 'Bodentemperatur',
        'Licht', 'UV_Index', 'PM25', 'PM10', 'station'
    ]

    # Insert the DataFrame into the SQLite table
    df.to_sql('WeatherData', conn, if_exists='append', index=False)

    conn.close()

# Create database if it doesnt exist
if not os.path.isfile(db_name):
    create_database()

In [11]:
# Todo find way to mark if its already finished
# 2023-12-06 fehlt im archiv

for day_folder in tqdm(os.listdir("./Data")):
    day = os.path.join("./Data", day_folder)
    # checking if it is a file
    if os.path.isfile(day):
        continue

    if os.path.isfile(os.path.join(day, data_file_name)):
        continue
    
    if not os.listdir(day):
        print(day, "date had no values")
        continue

    weather_station_day_data = []
    for weather_station in os.listdir(day):
        weather_data = os.path.join(day, weather_station)
        files = [os.path.join(weather_data, f) for f in os.listdir(weather_data) if (f.endswith(".csv") or f.endswith(".json"))]
        assignment = [f for f in files if f.endswith(".json")][0]
        files = [f for f in files if f.endswith(".csv")]
        measurements = get_assignment(assignment, files)
        data_frames = []
        for measurement in measurements.values():
            df = pd.read_csv(measurement['file'])
            df = df.rename(columns={'value': measurement['title']})
            data_frames.append(df)

        df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['createdAt'],
                                        how='outer'), data_frames)

        df_merged['station'] = weather_station_dict[weather_station]
        df_merged = df_merged.rename(columns=renaming_dict)
        weather_station_day_data.append(df_merged)
    
    df_concat = pd.concat(weather_station_day_data)
    df_concat = df_concat.rename(columns=column_dict)
    insert_dataframe_to_db(df_concat, db_name)
    df_concat.to_csv(os.path.join(day, data_file_name), index=False)

 68%|██████▊   | 434/634 [02:09<01:11,  2.79it/s]

./Data\2023-12-06 date had no values


 72%|███████▏  | 455/634 [02:12<00:15, 11.35it/s]

./Data\2023-12-25 date had no values


 73%|███████▎  | 463/634 [02:13<00:15, 10.76it/s]

...

 94%|█████████▍| 597/634 [02:35<00:07,  5.14it/s]

./Data\2024-05-17 date had no values
./Data\2024-05-18 date had no values


 95%|█████████▌| 604/634 [02:37<00:08,  3.43it/s]

./Data\2024-05-24 date had no values
./Data\2024-05-25 date had no values


100%|██████████| 634/634 [02:47<00:00,  3.80it/s]


In [12]:
df_concat

,createdAt,Temperatur,rel_Luftfeuchte,Luftdruck,Bodenfeuchte,Bodentemperatur,Licht,UV_Index,PM25,PM10,station
0,2024-06-22T00:00:23.302Z,13.19,94.68,1017.61,19.98,15.74,0.0,0.0,12.1,6.6,WetterBrg
1,2024-06-22T00:01:23.288Z,13.22,94.88,1017.58,20.41,21.54,0.0,0.0,12.8,8.2,WetterBrg
2,2024-06-22T00:02:23.281Z,13.25,95.39,1017.58,20.03,8.33,0.0,0.0,12.1,8.2,WetterBrg
3,2024-06-22T00:03:23.420Z,13.30,95.18,1017.63,20.84,23.80,0.0,0.0,11.1,7.1,WetterBrg
4,2024-06-22T00:04:23.427Z,13.31,95.39,1017.61,20.09,5.75,0.0,0.0,13.6,7.8,WetterBrg
...,...,...,...,...,...,...,...,...,...,...,...
1433,2024-06-22T23:55:20.649Z,15.92,90.40,982.90,11.33,22.51,0.0,0.0,2.1,5.7,PoS
1434,2024-06-22T23:56:20.655Z,15.90,90.40,982.89,11.12,16.39,0.0,0.0,1.6,4.0,PoS
1435,2024-06-22T23:57:20.631Z,15.92,90.60,982.89,11.12,17.03,0.0,0.0,1.9,4.6,PoS
1436,2024-06-22T23:58:20.628Z,15.92,90.60,982.89,11.12,16.39,0.0,0.0,1.7,4.4,PoS


In [14]:
"""Fetch a limited number of rows from the specified table in the SQLite database."""
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

cursor.execute(f"SELECT * FROM WeatherData LIMIT {3}")
rows = cursor.fetchall()

conn.close()
rows

[('2022-09-28T00:00:40.694Z',
  7.96,
  99.99,
  998.76,
  26.0,
  17.68,
  0.0,
  0.0,
  6.6,
  1.4,
  'WetterBrg'),
 ('2022-09-28T00:01:40.707Z',
  7.95,
  99.99,
  998.82,
  26.1,
  10.26,
  0.0,
  0.0,
  3.1,
  1.3,
  'WetterBrg'),
 ('2022-09-28T00:02:40.751Z',
  7.95,
  99.99,
  998.82,
  25.78,
  10.59,
  0.0,
  0.0,
  3.4,
  1.3,
  'WetterBrg')]

In [7]:
for foldername, subfolders, filenames in os.walk("./Data"):
        for filename in filenames:
            if filename == 'data.csv':
                file_path = os.path.join(foldername, filename)
                try:
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Failed to delete {file_path}: {e}")

Deleted: ./Data\2022-09-28\data.csv
Deleted: ./Data\2022-09-29\data.csv
Deleted: ./Data\2022-09-30\data.csv
Deleted: ./Data\2022-10-01\data.csv
Deleted: ./Data\2022-10-02\data.csv
Deleted: ./Data\2022-10-03\data.csv
Deleted: ./Data\2022-10-04\data.csv
Deleted: ./Data\2022-10-05\data.csv
Deleted: ./Data\2022-10-06\data.csv
Deleted: ./Data\2022-10-07\data.csv
Deleted: ./Data\2022-10-08\data.csv
Deleted: ./Data\2022-10-09\data.csv
Deleted: ./Data\2022-10-10\data.csv
Deleted: ./Data\2022-10-11\data.csv
Deleted: ./Data\2022-10-12\data.csv
Deleted: ./Data\2022-10-13\data.csv
Deleted: ./Data\2022-10-14\data.csv
Deleted: ./Data\2022-10-15\data.csv
Deleted: ./Data\2022-10-16\data.csv
Deleted: ./Data\2022-10-17\data.csv
Deleted: ./Data\2022-10-18\data.csv
Deleted: ./Data\2022-10-19\data.csv
Deleted: ./Data\2022-10-20\data.csv
Deleted: ./Data\2022-10-21\data.csv
Deleted: ./Data\2022-10-22\data.csv
Deleted: ./Data\2022-10-23\data.csv
Deleted: ./Data\2022-10-24\data.csv
Deleted: ./Data\2022-10-25\d